In [43]:
from sympy import *
from IPython.display import display, Math

In [44]:
c = symbols('c', nonzero=True, real=True, infinite=False)
c1 = symbols('c_1', nonzero=True, real=True, infinite=False)
c2 = symbols('c_2', nonzero=True, real=True, infinite=False)
v = symbols('v', nonzero=True, real=True, infinite=False) #d was replaced with v for clarity. (v = normalize(pixel position - camera position))
sigma_t = symbols('sigma_t', nonzero=True, real=True, infinite=False)
o = symbols('o', real=True, infinite=False)
u = symbols('u', real=True)
t = symbols('t', real=True)
t2 = symbols('t_2', real=True)
D = symbols('D', real=True, infinite=False, positive=True)
a = symbols('a', real=True, infinite=False, positive=True)
b = symbols('b', real=True, infinite=False, positive=True)

In [45]:
def heightFactor(y):
    return exp(-y/c)

The outer exponential of\
$ e^{- \int_{0}^{D}{\sigma_t(t) \mathrm{d}t}} $\
is not relevant to solving the integral and will be omitted in the following

Only height based

In [46]:
i = Integral(sigma_t*heightFactor(o+t*v), (t,0,D))
i

Integral(sigma_t*exp((-o - t*v)/c), (t, 0, D))

In [47]:
i.doit()

-c*sigma_t*exp((-D*v - o)/c)/v + c*sigma_t*exp(-o/c)/v

or in the case that v=0:

In [48]:
D*sigma_t*exp(-o/c)

D*sigma_t*exp(-o/c)

Adding a camera fade by scaling the "intensity" of the fog with a factor of $0$ to $1$ between distances $a$ and $b$ to the camera using a weighting factor $w(\mathrm{depth})$\
(which is of course not physical at all)

In [49]:
w = Function('w')

When integrating over $o+tv$, $t$ is already the distance from the camera since $v$ is normalized

In [50]:
generalIntegral = Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,0,D))
generalIntegral

Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, 0, D))

which is the same as:

In [51]:
generalIntegral = Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,0,a)) + Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,a,b)) + Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,b,D))
generalIntegral

Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, 0, a)) + Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, a, b)) + Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, b, D))

Where by definition of the fade, $w(t)$ is $0$ in the bounds of the first integral and $1$ in the bounds of the last integral.
This simplifies to 

In [52]:
generalIntegral = Integral(w(t)*sigma_t*heightFactor(o+t*v), (t,a,b)) + Integral(1*sigma_t*heightFactor(o+t*v), (t,b,D))
generalIntegral

Integral(sigma_t*exp((-o - t*v)/c), (t, b, D)) + Integral(sigma_t*w(t)*exp((-o - t*v)/c), (t, a, b))

Solving the last part:

In [53]:
endIntegral = Integral(sigma_t*heightFactor(o+t*v), (t,b,D))
endIntegral

Integral(sigma_t*exp((-o - t*v)/c), (t, b, D))

In [54]:
endIntegral.doit()

-c*sigma_t*exp((-D*v - o)/c)/v + c*sigma_t*exp((-b*v - o)/c)/v

Or in case $v=0$

In [55]:
simplify(endIntegral.subs(v,0).doit())

sigma_t*(D - b)*exp(-o/c)

Solving the middle integral depends on the choice of fade function

Simple linear ramp:

In [56]:
def linearRamp(x, start, end):
    return ((x - start) / (end - start))

In [57]:
middleIntegralLinear = Integral(linearRamp(t,a,b)*sigma_t*heightFactor(o+t*v), (t,a,b))
middleIntegralLinear

Integral(sigma_t*(-a + t)*exp((-o - t*v)/c)/(-a + b), (t, a, b))

In [58]:
middleIntegralLinear.doit()

Piecewise((-c**2*sigma_t*exp((-a*v - o)/c)/(a*v**2 - b*v**2) + (-a*c*sigma_t*v + b*c*sigma_t*v + c**2*sigma_t)*exp((-b*v - o)/c)/(a*v**2 - b*v**2), Ne(a*v**2 - b*v**2, 0)), (a**2*sigma_t/(2*a - 2*b) - a**2*sigma_t/(a - b) + a*b*sigma_t/(a - b) - b**2*sigma_t/(2*a - 2*b), True))

In [70]:
simplify(middleIntegralLinear.doit().args[0][0])
# simplify(middleIntegralLinear.doit())

c*sigma_t*(-c*exp(-(a*v + o)/c) + (-a*v + b*v + c)*exp(-(b*v + o)/c))/(v**2*(a - b))

We can safely assume that $a\neq b$ as that would not be a valid fade\
$v=0$ on the other hand can happen, but solving with that in mind is simple since the area under the linear ramp from a to b is simply $(b-a)/2$ 

In [60]:
simplify(middleIntegralLinear.subs(v,0).doit())

sigma_t*(-a + b)*exp(-o/c)/2

Another exception occurs when the pixel depth is lower than b, in that case the middle integral is bounded by the pixels depth instead

In [61]:
middleIntegralLinear2 = Integral(linearRamp(t,a,b)*sigma_t*heightFactor(o+t*v), (t,a,D))
middleIntegralLinear2

Integral(sigma_t*(-a + t)*exp((-o - t*v)/c)/(-a + b), (t, a, D))

In [75]:
simplify(middleIntegralLinear2.doit().args[0][0])
# middleIntegralLinear2.doit()

c*sigma_t*(-c*exp(-(a*v + o)/c) + (D*v - a*v + c)*exp(-(D*v + o)/c))/(v**2*(a - b))

In [62]:
simplify(middleIntegralLinear2.subs(v,0).doit())

sigma_t*(-D**2 + 2*D*a - a**2)*exp(-o/c)/(2*(a - b))